In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece


In [ ]:
!nvidia-smi

Thu Jul  8 05:44:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Imports

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
import pandas as pd
from collections import namedtuple
from sklearn import preprocessing
from bert import bert_tokenization
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.5.0
Hub version:  0.12.0


##Reading Datasets and droping nan values

In [ ]:
df_train = pd.read_excel('/content/drive/MyDrive/Datasets/Tamil__hasoc_train.xlsx',names=["ID","Tweets","Labels"])
df_train.dropna(inplace=True)
df_train.reset_index(drop=True, inplace=True)
df_val = pd.read_csv("/content/drive/MyDrive/Datasets/Tamil_hasoc_dev.tsv", sep="\t",names=["ID","Tweets","Labels"])
df_val.dropna(inplace=True)
df_val.reset_index(drop=True, inplace=True)

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/Datasets/tamil__hasoc_test.tsv", sep="\t",names=["ID","Tweets"])
# df_test.dropna(inplace=True)
# df_test.reset_index(drop=True, inplace=True)
df_test.shape

(1001, 2)

In [ ]:
df_val.head()

,ID,Tweets,Labels
0,TA_TW15946,Take it this thevidiya Kandipa indha page admi...,OFF
1,TA_TW10175,enga veetla itha nadakum Athum oru varushama t...,NOT
2,TA_TW15947,"Indha Sallli Punda, Dummy Pundalam Orama Iruka...",OFF
3,TA_TW15174,Juriya poola tier 1 la umburan tha kulla punda...,OFF
4,TA_TW15182,Kullans lam umba therila Loosu kuthi maari umb...,OFF


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train.head(10)

,ID,Tweets,Labels
0,TA_HL101,@Asha Apo neenga atha government ku theriya pa...,NOT
1,TA_HL102,@Bala sundar ayyo sorry...antha line ah explai...,NOT
2,TA_HL105,@kalimuthu ne ena lusa...yaaru edhu panaalum e...,NOT
3,TA_HL109,1st baby ku neat ah feed panunga plzz ipdi iru...,NOT
4,TA_HL113,"2012 il vazhgiromnu iruku ithula, apdina?",NOT
5,TA_HL118,30 varusa kadan. 25 age engayo idikuthe,NOT
6,TA_HL124,a vanitha veliya po ethuku thirumpi vantha,NOT
7,TA_HL125,à®•à¯à®´à®¨à¯à®¤à¯ˆ ga taste ah saptanum nu ...,NOT
8,TA_HL127,Aaiiii Jolly Yellam onnah polam onnah polam oa...,NOT
9,TA_HL128,"aaluku etha mathri pesarathu,thurumbi vanthu p...",NOT


In [ ]:
df_test.head(10)

,ID,Tweets
0,Tam_1,Indha movie ku award tharlana avanga mansanay ...
1,Tam_2,kritheeck Kookaburra en unaku enachu? Cbsc ah??
2,Tam_3,Actually Oru particular bus incident thalaiya ...
3,Tam_4,Small suggestions: mic ah shirt la pottukunga bro
4,Tam_5,Karnan padathulaa oru pombaa varumlaa athuu en...
5,Tam_6,Madhanu konjam gunda agita mathanu odamba fit ...
6,Tam_7,thambi oru chinna correction .. odukkappattava...
7,Tam_8,Thala weight potuteenga
8,Tam_9,thambi oru chinna correction .. odukkappattava...
9,Tam_10,Odukapatta jathi nu soluvathu thavaru.... Oru ...


##Preprocessing





In [ ]:
import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))
    
def preprocessing(document):
  document = str(document)
  # document = re.sub(r'[0-9]', ' ', document)
  document = document.replace("@"," ")
  # document = regex.sub(' ', document)
  # document = document.lower()
  # document = re.sub(' +', ' ', document)
  tokens = document.split()
  preprocessed_text = ' '.join(tokens)
  return preprocessed_text
  

In [ ]:
df_train.Tweets =  df_train.Tweets.apply(preprocessing)
df_val.Tweets =  df_val.Tweets.apply(preprocessing)
df_test.Tweets =  df_test.Tweets.apply(preprocessing)

In [ ]:
df_train.head(10)

,ID,Tweets,Labels
0,TA_HL101,Asha Apo neenga atha government ku theriya pad...,NOT
1,TA_HL102,Bala sundar ayyo sorry...antha line ah explain...,NOT
2,TA_HL105,kalimuthu ne ena lusa...yaaru edhu panaalum en...,NOT
3,TA_HL109,1st baby ku neat ah feed panunga plzz ipdi iru...,NOT
4,TA_HL113,"2012 il vazhgiromnu iruku ithula, apdina?",NOT
5,TA_HL118,30 varusa kadan. 25 age engayo idikuthe,NOT
6,TA_HL124,a vanitha veliya po ethuku thirumpi vantha,NOT
7,TA_HL125,à®•à¯à®´à®¨à¯à®¤à¯ˆ ga taste ah saptanum nu ...,NOT
8,TA_HL127,Aaiiii Jolly Yellam onnah polam onnah polam oa...,NOT
9,TA_HL128,"aaluku etha mathri pesarathu,thurumbi vanthu p...",NOT


##Mapping the labels correctly 

In [ ]:
df_train.Labels = df_train.Labels.map({'not': 'NOT', 'OFf': 'OFF','NOT': 'NOT', 'OFF': 'OFF'})

In [ ]:
df_train.head(10)

,ID,Tweets,Labels
0,TA_HL101,Asha Apo neenga atha government ku theriya pad...,NOT
1,TA_HL102,Bala sundar ayyo sorry...antha line ah explain...,NOT
2,TA_HL105,kalimuthu ne ena lusa...yaaru edhu panaalum en...,NOT
3,TA_HL109,1st baby ku neat ah feed panunga plzz ipdi iru...,NOT
4,TA_HL113,"2012 il vazhgiromnu iruku ithula, apdina?",NOT
5,TA_HL118,30 varusa kadan. 25 age engayo idikuthe,NOT
6,TA_HL124,a vanitha veliya po ethuku thirumpi vantha,NOT
7,TA_HL125,à®•à¯à®´à®¨à¯à®¤à¯ˆ ga taste ah saptanum nu ...,NOT
8,TA_HL127,Aaiiii Jolly Yellam onnah polam onnah polam oa...,NOT
9,TA_HL128,"aaluku etha mathri pesarathu,thurumbi vanthu p...",NOT


In [ ]:
df_val.Labels.unique()

array(['OFF', 'NOT'], dtype=object)

##Label Encoding to 0 and 1

In [ ]:
unique_labels = list(np.unique(df_train["Labels"]))

train_x = df_train["Tweets"].values
train_y = df_train["Labels"].values

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
# le = preprocessing.LabelEncoder()

train_y = le.fit_transform(train_y)
train_y = tf.keras.utils.to_categorical(train_y, num_classes=len(unique_labels), dtype='float32')

val_x = df_val["Tweets"].values
val_y = df_val["Labels"].values

val_y = le.fit_transform(val_y)
val_y = tf.keras.utils.to_categorical(val_y, num_classes=len(unique_labels), dtype='float32')


print("number of unique labels", len(unique_labels))

test_x = df_test["Tweets"].values

number of unique labels 2


##Helper Functions

In [ ]:
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

# Function to create attention masks
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

# Function to create segment ids
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

# Function to create input_ids, attention_masks, segment_ids for sample
def create_single_input(sentence,MAX_LEN, MAX_SEQ_LEN):
  
  stokens = tokenizer.tokenize(sentence)
  
  stokens = stokens[:MAX_LEN]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)

  return ids,masks,segments

def create_input_array(sentences, MAX_SEQ_LEN):

  input_ids, input_masks, input_segments = [], [], []

  for sentence in tqdm(sentences,position=0, leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2, MAX_SEQ_LEN)

    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

##Downloading the MuRIL model from TFHub

In [ ]:
muril_layer = hub.KerasLayer("https://tfhub.dev/google/MuRIL/1", trainable=True)

# Create tokenizer
vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
max_seq_len = 100
train_x = create_input_array(train_x, max_seq_len)
val_x = create_input_array(val_x, max_seq_len)
test_x = create_input_array(test_x, max_seq_len)

100%|██████████| 1001/1001 [00:00<00:00, 3650.12it/s]


##Defining the F1 metric

In [ ]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
df_train.shape

(3999, 3)

##Establishing class weights

In [ ]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight('balanced',
                                             np.unique(df_train['Labels']),
                                             df_train['Labels']))

weights={}
for index, weight in enumerate(class_weights) :
  weights[index]=weight

print(weights)

{0: 0.9903417533432393, 1: 1.0098484848484848}


In [ ]:
#downloading model

In [ ]:
pip install tf-models-official

     |████████████████████████████████| 1.6MB 9.6MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 686kB 51.2MB/s 
     |████████████████████████████████| 358kB 56.0MB/s 
     |████████████████████████████████| 215kB 58.6MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 645kB 41.6MB/s 
     |████████████████████████████████| 38.2MB 65kB/s 
     |████████████████████████████████| 102kB 15.3MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=f108549dca458e044c476efa18d1675e04c5a078ef41939affafadd3e39f4402
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22258 sha256=f1c8db1652fd858bffa3346ac5a08e6121d275fd5661284a60dbad77511d325d
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e

##Defining the model

In [ ]:
input_word_ids = tf.keras.layers.Input(shape=(100,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(100,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(100,), dtype=tf.int32,
                                    name="segment_ids")
  
outputs = muril_layer(dict(input_word_ids = input_word_ids, input_mask = input_mask, input_type_ids = segment_ids))
x = tf.keras.layers.Dropout(0.2)(outputs["pooled_output"]) # take pooled output layer
final_output = tf.keras.layers.Dense(2, activation="sigmoid", name="dense_output")(x)

model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=final_output)

    
#   optimizer = 
model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  metrics=['accuracy',f1_m])




In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'sequence_output':  237556225   input_mask[0][0]                 
                                                                 segment_ids[0][0]            

##Making checkpoint

In [ ]:
metric = 'val_f1_m'
model_checkpoint_callback  = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/Datasets/tam3", monitor=metric,
                    verbose=2, save_best_only=True, mode='max')

###Skipping the training as loading pretrained

In [ ]:
# num_epochs = 15

# # Get the model object
# history = model.fit(train_x, train_y, epochs = num_epochs, batch_size = 50, validation_data = (val_x, val_y),class_weight=weights,callbacks=[model_checkpoint_callback])

In [ ]:
# from sklearn.metrics import classification_report
# preds = model.predict(val_x)>0.5
# print(classification_report(val_y, preds))

In [ ]:
#Reloading the model with best validation F1 score

model2 = tf.keras.models.load_model('/content/drive/MyDrive/Datasets/tam3', custom_objects={'f1_m':f1_m})

In [ ]:
from sklearn.metrics import classification_report
preds = model2.predict(val_x)>0.5
print(classification_report(val_y, preds))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91       465
           1       0.92      0.91      0.91       475

   micro avg       0.91      0.91      0.91       940
   macro avg       0.91      0.91      0.91       940
weighted avg       0.91      0.91      0.91       940
 samples avg       0.91      0.91      0.91       940



In [ ]:
# model2.save('/content/drive/MyDrive/Datasets/tahsoc91')

In [ ]:
# preds_test = model2.predict(test_x)>0.5

In [ ]:
# labels = []

# for i in preds_test:
#   if i[0] == True:
#     labels.append("NOT")
#   elif i[1] == True:
#     labels.append("OFF")

In [ ]:
# df_test["Label"] = labels

In [ ]:
# df_test.Label.value_counts()/df_test.shape[0]*100

NOT    69.330669
OFF    30.669331
Name: Label, dtype: float64

In [ ]:
# df_test.head(10)

,ID,Tweets,Label
0,Tam_1,Indha movie ku award tharlana avanga mansanay ...,NOT
1,Tam_2,kritheeck Kookaburra en unaku enachu? Cbsc ah??,OFF
2,Tam_3,Actually Oru particular bus incident thalaiya ...,NOT
3,Tam_4,Small suggestions: mic ah shirt la pottukunga bro,NOT
4,Tam_5,Karnan padathulaa oru pombaa varumlaa athuu en...,NOT
5,Tam_6,Madhanu konjam gunda agita mathanu odamba fit ...,NOT
6,Tam_7,thambi oru chinna correction .. odukkappattava...,NOT
7,Tam_8,Thala weight potuteenga,OFF
8,Tam_9,thambi oru chinna correction .. odukkappattava...,NOT
9,Tam_10,Odukapatta jathi nu soluvathu thavaru.... Oru ...,NOT


In [ ]:
# !nvidia-smi


In [ ]:
# df_test.to_csv("submission5.tsv", index=False,sep="\t")

In [ ]:
# metric = 'val_f1_m'
# model_checkpoint_callback2  = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/Datasets/tam2", monitor=metric,
#                     verbose=2, save_best_only=True, mode='max')

In [ ]:
# model2.compile(loss='categorical_crossentropy',
#                   optimizer=tf.keras.optimizers.Adam(learning_rate=1e-7),
#                   metrics=['accuracy',f1_m])

# num_epochs = 15

# # Get the model object
# history = model2.fit(train_x, train_y, epochs = num_epochs, batch_size = 50, validation_data = (val_x, val_y),class_weight=weights,callbacks=[model_checkpoint_callback2])

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Epoch 1/15
80/80 [==============================] - 108s 1s/step - loss: 0.0886 - accuracy: 0.9800 - f1_m: 0.9800 - val_loss: 0.4652 - val_accuracy: 0.9053 - val_f1_m: 0.9053

Epoch 00001: val_f1_m improved from -inf to 0.90526, saving model to /content/drive/MyDrive/Datasets/tam2


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Datasets/tam2/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Datasets/tam2/assets


Epoch 2/15
62/80 [======================>.......] - ETA: 19s - loss: 0.0859 - accuracy: 0.9819 - f1_m: 0.9819